In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA, KernelPCA
from config import *
import torch

In [ ]:
projects = {}
for folder in sorted(os.listdir(JAVA_FILES_PATH)):
    if '.csv' in folder or '.ipynb' in folder or '.txt' in folder or folder in projects_to_exclude: continue
    folder_path = JAVA_FILES_PATH + "/" + folder
    for version in sorted(os.listdir(folder_path),reverse=True):
        if '.ipynb' in version: continue
        version_representation = []
        version_path = folder_path + "/" + version
        pickle_path = version_path + f"/{OUTPUT_PATH}"
        for pickle_file in os.listdir(pickle_path):
            file = open(pickle_path + "/" + pickle_file,'rb')
            p = pickle.load(file)
            np_array = p.cpu().detach().numpy().flatten()
            version_representation.append(np_array)
        if version_representation == []:
            continue
        df = pd.DataFrame(version_representation)
        projects[folder+"_"+version] = df.mean()

In [ ]:
db = pd.DataFrame(projects).T
features = db.fillna(0)
features = features.reset_index()
features['pv'] = features['index']
features = features.drop(columns=['index'])
features.set_index('pv',inplace=True)
features.to_csv(OUTPUT_EMBEDDING_FEATURES)